# Multi-label classification of racial tweets:

In [18]:
#Read in csv of cleaned data
df = pd.read_csv('df_tweets.csv', index_col=0)

#Create training set

df_train = df[df['tweet'].notnull()]
print(df_train.shape)
print('')
print(df_train.columns)
df_train.head()

(6481, 5)

Index(['tweetidg', 'tweet', 'positive', 'negative', 'neutral'], dtype='object')


,tweetidg,tweet,positive,negative,neutral
0,588687492888551424g,i am the type of nigga tryna get rich,0.0,0.0,1.0
1,592553981601304576g,sheblasiannn smuckers,0.0,0.0,1.0
2,717371522956984320g,proteinwisdom we even make japanese cars and e...,0.0,0.0,1.0
3,590292541125328898g,i wanted to see my nigga when i went back home...,0.0,1.0,0.0
4,592522044480225282g,wth stevo is that a nigga,0.0,0.0,1.0


In [11]:
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

### Create a table to examine the distribution of tags

In [12]:

df_sparse = df_train.drop(['tweetidg', 'tweet'], axis=1)

counts = []
priors = []
categories = list(df_sparse.columns.values)
for i in categories:
    numarts = df_sparse[i].sum()
    counts.append((i, numarts))

df_stats = pd.DataFrame(counts, columns=['Category', 'Number_of_Tweets'])
total = df_stats.Number_of_Tweets.sum()

df_stats['priors'] = df_stats['Number_of_Tweets']/total

print(df_stats)
print('')
print('Total number of labels:  ', total)

   Category  Number_of_Tweets    priors
0  positive            1642.0  0.253356
1  negative            1977.0  0.305046
2   neutral            2862.0  0.441599

Total number of labels:   6481.0


## Multi-label classification

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
df_train.columns

Index(['tweetidg', 'tweet', 'positive', 'negative', 'neutral'], dtype='object')

In [15]:
#Make sure all entries in df_matrix['text'] are strings
df_train['tweet'] = df_train['tweet'].astype(str)


from sklearn.feature_extraction.text import CountVectorizer

#Function for positive labels
def make_x(df, vectorizer=None):
    if vectorizer is None:
        vectorizer = CountVectorizer(stop_words='english',binary=False,max_df=50,
                                     min_df=1,strip_accents='unicode',max_features=200)
    X = vectorizer.fit_transform(df_train.tweet)
    
    #X = X.tocsc()  # some versions of sklearn return COO format
    return type(X)
#Call the function: X
make_x(df_train)



#Function for negative labels
def make_xy_neg(df, vectorizer=None):
    if vectorizer is None:
        vectorizer = CountVectorizer(stop_words='english',binary=False,max_df=25,
                                     min_df=1,strip_accents='unicode',max_features=100)
    X = vectorizer.fit_transform(df_train.tweet)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = df_train['negative'].values.astype(np.int)
    return X, y



#Function for neutral labels
def make_xy_neut(df, vectorizer=None):
    if vectorizer is None:
        vectorizer = CountVectorizer(stop_words='english',binary=False,max_df=25,
                                     min_df=1,strip_accents='unicode',max_features=100)
    X = vectorizer.fit_transform(df_train.tweet)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = df_train['neutral'].values.astype(np.int)
    return X


## Multinomial Naive Bayes

In [20]:
vectorizer = CountVectorizer(stop_words='english',binary=False,max_df=50,
                                     min_df=1,strip_accents='unicode',max_features=200)
X = vectorizer.fit_transform(df_train.tweet)
    

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

clf_NB = MultinomialNB()

#Positive scores
yp = df_train['positive'].values.astype(np.int)
X_trainp, X_testp, y_trainp, y_testp = train_test_split(X, yp, test_size=0.30, random_state=42)

clf_NB.fit(X_trainp,y_trainp)

train_accuracy_pos =clf_NB.score(X_trainp,y_trainp)
test_accuracy_pos = clf_NB.score(X_testp, y_testp)
print("Positive - training set accuracy: ", train_accuracy_pos)
print("Positive - test set accuracy: ", test_accuracy_pos, '\n')
        
#Negative scores
yn = df_train['negative'].values.astype(np.int)
X_trainn, X_testn, y_trainn, y_testn = train_test_split(X, yn, test_size=0.30, random_state=42)

clf_NB.fit(X_trainn,y_trainn)

train_accuracy_neg = clf_NB.score(X_trainn, y_trainn)
test_accuracy_neg = clf_NB.score(X_testn, y_testn)
print("Negative - training set accuracy: ", train_accuracy_neg)
print("Negative - test set accuracy: ", test_accuracy_neg, '\n')
        
#Neutral scores
yne = df_train['neutral'].values.astype(np.int)

X_train, X_test, y_train, y_test = train_test_split(X, yne, test_size=0.30, random_state=42)

clf_NB.fit(X_train,y_train)

train_accuracy_neut = clf_NB.score(X_train, y_train)
test_accuracy_neut = clf_NB.score(X_test, y_test)
print("Neutral - training set accuracy: ", train_accuracy_neut)
print("Neutral - test set accuracy: ", test_accuracy_neut)   

NameError: name 'X' is not defined

This is looking good! But accuracy can be deceiving. High accuracy scores can still happen if your model detects many true negatives, but no try positives! Let's check:

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score, f1_score


#Prediciton with NB
conmatrxNB = []
precisionNB = []
recallNB = []
f1NB = []


for category in categories:

    y_train_pred = cross_val_predict(clf_NB, X, df_train[category], cv=5)
    
    cmNB = confusion_matrix(df_train[category], y_train_pred)
    conmatrxNB.append(cmNB)
    precNB = precision_score(df_train[category], y_train_pred)    
    precisionNB.append(precNB)
    recNB = recall_score(df_train[category], y_train_pred)
    recallNB.append(recNB)
    fNB = f1_score(df_train[category], y_train_pred)
    f1NB.append(fNB)

    
dictNB = {'Categories':categories, 'Confusion Matrix':conmatrxNB, 'Precision_NB':precisionNB, 'Recall':recallNB, 'F1':f1NB}
df_NB = pd.DataFrame(dictNB) 
#df_NB.to_csv('Results_NB_tweets.csv')

df_NB_tweets = df_NB[df_NB['Precision_NB'] >= 0]
 
df_NB_tweets

As I expected, high accuracy doesn't always mean high precision or recall. We can see that these are lower. I will run the other models first, then tune parameters on the one showing the most promise for this data. Note that for the neutral label, accuracy was lower than precision. For this study, we want to decrease the probability of a Type 2 error - a false positive where we say there is an effect but there is not. Precision is the best score to tune because it is a measure of how sure we are that a label is a true label (true positive) if our algorithm classifies it as such. After running the other two classifiers, I will tune the hyper-parameters for the best performing classifier at these default settings. 

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

clf_SVC = OneVsRestClassifier(LinearSVC(), n_jobs=1)

#Positive scores
yp = df_train['positive'].values.astype(np.int)
X_trainp, X_testp, y_trainp, y_testp = train_test_split(X, yp, test_size=0.30, random_state=42)

clf_SVC.fit(X_trainp,y_trainp)

train_accuracy_pos =clf_SVC.score(X_trainp,y_trainp)
test_accuracy_pos = clf_SVC.score(X_testp, y_testp)
print("Positive - training set accuracy: ", train_accuracy_pos)
print("Positive - test set accuracy: ", test_accuracy_pos, '\n')
        
#Negative scores
yn = df_train['negative'].values.astype(np.int)
X_trainn, X_testn, y_trainn, y_testn = train_test_split(X, yn, test_size=0.30, random_state=42)

clf_SVC.fit(X_trainn,y_trainn)

train_accuracy_neg = clf_SVC.score(X_trainn, y_trainn)
test_accuracy_neg = clf_SVC.score(X_testn, y_testn)
print("Negative - training set accuracy: ", train_accuracy_neg)
print("Negative - test set accuracy: ", test_accuracy_neg, '\n')
        
#Neutral scores
yne = df_train['neutral'].values.astype(np.int)

X_train, X_test, y_train, y_test = train_test_split(X, yne, test_size=0.30, random_state=42)

clf_SVC.fit(X_train,y_train)

train_accuracy_neut = clf_SVC.score(X_train, y_train)
test_accuracy_neut = clf_SVC.score(X_test, y_test)
print("Neutral - training set accuracy: ", train_accuracy_neut)
print("Neutral - test set accuracy: ", test_accuracy_neut)   

## LinearSVC

In [ ]:
clf_SVC = LinearSVC()


#Prediciton with NB
conmatrxSVC = []
precisionSVC = []
recallSVC = []
f1SVC = []



for category in categories:

    y_train_pred = cross_val_predict(clf_SVC, X, df_train[category], cv=5)
    
    cmSVC = confusion_matrix(df_train[category], y_train_pred)
    conmatrxSVC.append(cmSVC)
    precSVC = precision_score(df_train[category], y_train_pred)    
    precisionSVC.append(precSVC)
    recSVC = recall_score(df_train[category], y_train_pred)
    recallSVC.append(recSVC)
    fSVC = f1_score(df_train[category], y_train_pred)
    f1SVC.append(fSVC)

    
dictSVC = {'Categories':categories, 'Confusion Matrix':conmatrxSVC, 'Precision_SVC':precisionSVC, 
           'Recall':recallSVC, 'F1':f1SVC}
df_SVC = pd.DataFrame(dictSVC) 
#df_NB.to_csv('Results_NB_tweets.csv')

df_SVC_tweets = df_SVC[df_SVC['Precision_SVC'] >= 0]
 
df_SVC_tweets

## Logistic Regression

In [ ]:
clf_LR = LogisticRegression()


#Prediciton with NB
conmatrxLR = []
precisionLR = []
recallLR = []
f1LR = []



for category in categories:

    y_train_pred = cross_val_predict(clf_LR, X, df_train[category], cv=5)
    
    cmLR = confusion_matrix(df_train[category], y_train_pred)
    conmatrxLR.append(cmLR)
    precLR = precision_score(df_train[category], y_train_pred)    
    precisionLR.append(precLR)
    recLR = recall_score(df_train[category], y_train_pred)
    recallLR.append(recLR)
    fLR = f1_score(df_train[category], y_train_pred)
    f1LR.append(fLR)

    
dictLR = {'Categories':categories, 'Confusion Matrix':conmatrxLR, 'Precision_LR':precisionLR, 
           'Recall':recallLR, 'F1':f1LR}
df_LR = pd.DataFrame(dictLR) 
#df_NB.to_csv('Results_NB_tweets.csv')

df_LR_tweets = df_LR[df_LR['Precision_LR'] >= 0]
 
df_LR_tweets

In [ ]:
df_cat = df_NB.Categories
df_prec_NB  =df_NB.Precision_NB
df_prec_SVC  = df_SVC.Precision_SVC
df_prec_LR  = df_LR.Precision_LR

df_prec1 = pd.DataFrame(columns = ['Categories', 'Precision_NB', 'Precision_SVC', 
                                   'Precision_LR', 'Max_Precision'])
df_prec1.Categories = df_NB.Categories
df_prec1.Precision_NB = df_NB.Precision_NB
df_prec1.Precision_SVC  = df_SVC.Precision_SVC
df_prec1.Precision_LR  = df_LR.Precision_LR
df_prec1.Max_Precision = df_prec1.max(axis=1)
df_prec1

## Tune hyperparameters for multi

The ‘newton-cg’, ‘sag’ and ‘lbfgs’ solvers support only l2 penalties. I will first leave out the solver and check for the best penalty. The I will choose the solvers that best run with that penalty.



In [9]:
clf_LR = LogisticRegression()

 
#The ‘newton-cg’, ‘sag’ and ‘lbfgs’ solvers support only l2 penalties.

    # Create regularization penalty space
penalty = ['l1', 'l2']

    # Create regularization hyperparameter space
C = np.logspace(-4, 4, 20)

#solver = ['liblinear', 'sag', 'newton-cg', 'saga', 'lbfgs']


    # Create hyperparameter options
hyperparameters = dict(penalty = penalty, C=C)


     # Create grid search using 5-fold cross validation
clf_tune = GridSearchCV(clf_LR, hyperparameters, cv=5, verbose=0, scoring='f1_score')               

for category in categories:
    # Fit grid search
    best_model = clf_tune.fit(X, df_train[category])                
                
    # View best hyperparameters
    print(category)
    print('Best penalty:', best_model.best_estimator_.get_params()['penalty'])
    print('Best C:', best_model.best_estimator_.get_params()['C'],'\n')         


NameError: name 'GridSearchCV' is not defined

It is obvious that each label will need to be individually tuned. Let's check for the solver under the l2 penalty. We need not check l1 because liblinear is both the default and the only solver that will work with l1.

In [294]:

#The ‘newton-cg’, ‘sag’ and ‘lbfgs’ solvers support only l2 penalties.

    # Create regularization penalty space
penalty = ['l2']

    # Create regularization hyperparameter space
C = np.logspace(-4, 4, 20)

solver = ['liblinear', 'sag', 'newton-cg', 'saga', 'lbfgs']

    # Create hyperparameter options
hyperparameters = dict(penalty=penalty,C=C, solver=solver)


     # Create grid search using 5-fold cross validation
clf_tune = GridSearchCV(clf_LR, hyperparameters, cv=5, verbose=0, scoring='precision')               

for category in categories:
    # Fit grid search
    best_model = clf_tune.fit(X, df_train[category])                
                
    # View best hyperparameters
    print(category)
    print('Best solver:', best_model.best_estimator_.get_params()['solver'])
    print('Best C:', best_model.best_estimator_.get_params()['C'],'\n')         


positive
Best solver: liblinear
Best C: 0.23357214690901212 

negative
Best solver: liblinear
Best C: 0.08858667904100823 

neutral
Best solver: sag
Best C: 0.03359818286283781 



OK, we have a different combination for each label. I will run each separately and examine the differences before predicting on the rately and examine the differences before predicting on the unlabeled data.

## Train the tuned models

In [298]:
#hyperpos = ['C=0.23357214690901212']
#hyperneg = ['penalty=l2, C=0.08858667904100823']
#hyperneut = ["penalty=l2, C=0.03359818286283781, solver='sag'"]




#Prediciton with NB
conmatrxLR = []
precisionLR = []
recallLR = []
f1LR = []


for category in categories:
    if category == 'positive':
        clf_tuned = LogisticRegression(penalty='l1',C=0.23357214690901212,  solver='liblinear')
    if category == 'negative':
        clf_tuned = LogisticRegression(penalty='l2', C=0.08858667904100823, solver='liblinear')
    if category == 'neutral':
        clf_tuned = LogisticRegression(penalty='l2', C=0.03359818286283781, solver='sag')
    
    y_train_pred = cross_val_predict(clf_tuned, X, df_train[category], cv=5)
    
    cmLR = confusion_matrix(df_train[category], y_train_pred)
    conmatrxLR.append(cmLR)
    precLR = precision_score(df_train[category], y_train_pred)    
    precisionLR.append(precLR)
    recLR = recall_score(df_train[category], y_train_pred)
    recallLR.append(recLR)
    fLR = f1_score(df_train[category], y_train_pred)
    f1LR.append(fLR)

dictLR = {'Categories':categories, 'Confusion Matrix':conmatrxLR, 'Precision_LR':precisionLR, 
           'Recall':recallLR, 'F1':f1LR}
df_LR = pd.DataFrame(dictLR) 
#df_NB.to_csv('Results_NB_tweets.csv')

df_LR_tweets = df_LR[df_LR['Precision_LR'] >= 0]
 
df_LR_tweets

,Categories,Confusion Matrix,Precision_LR,Recall,F1
0,positive,"[[4835, 4], [1624, 18]]",0.818182,0.010962,0.021635
1,negative,"[[4501, 3], [1943, 34]]",0.918919,0.017198,0.033764
2,neutral,"[[3614, 5], [2827, 35]]",0.875000,0.012229,0.024121


In [300]:
 conmatrxLR[0]

array([[4835,    4],
       [1624,   18]])